* MQ Telemetry Transport, where the MQ came from an old IBM product line called IBM MQ for Message Queues

* MQTT is a lightweight publish/subscribe architecture that is designed for resource-constrained devices and low-bandwidth setups

* Port 1883 is reserved for use with MQTT



#Publish/Subscribe

* broker, or hub, that receives messages published to it by different clients.

* The **broker** then routes the messages to any clients subscribing to those particular messages.

* One **client** publishes to a topic, while another client **subscribes** to the topic.

* MQTT cannot publish a video frame 

Eg: a stop light classifier may publish on one topic, with an intermediate system that determines when to brake subscribing to that topic, and then that system could publish to another topic that the actual brake system itself subscribes to.

#Communicating with MQTT

* Python library for working with MQTT called `paho-mqtt`.

* sub-library called `client`, which create an MQTT client that can publish or subscribe to the broker. 

* To connect the client, and then begin to either publish or subscribe to topics
    1. Broker IP
    2. Broker port number

* **Publishing** involves feeding in the topic name, as well as a dictionary containing a message that is dumped to JSON
  
*  **Subscribing** just involves feeding in the topic name to be subscribed to



#API usage

https://pypi.org/project/paho-mqtt/ - documentation

In [0]:
import paho.mqtt.client as mqtt

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("$SYS/#")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("mqtt.eclipse.org", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

#Streaming Images to a Server

## FFMPEG: “fast forward” MPEG

* The `ffserver` feature of FFmpeg, which, similar to MQTT, will actually have an intermediate FFmpeg server that video frames are sent to.

* The final Node server that displays a webpage will actually get the video from that FFmpeg server.

* There are other ways to handle streaming video as well. In Python, you can also use a flask server to do some similar things, 

##Setting up FFmpeg

* Using ffserver in particular requires a configuration file 

* **config file ** sets the port and IP address of the server, as well as settings like the ports to receive video from, and the framerate of the video.

* These settings can also allow it to listen to the system stdout buffer, which is how you can send video frames to it in Python

##Sending frames to FFmpeg

`sys.stdout.buffer.write(frame)` and `sys.stdout.flush()` to send the frame to the `ffserver` when it is running.



In [0]:
#ffmpeg folder containing the configuration file
#launch the ffserver with the following from the command line:

sudo ffserver -f ./ffmpeg/server.conf

From there, you need to actually pipe the information from the Python script to FFmpeg. To do so, you add the | symbol after the python script (as well as being after any related arguments to that script, such as the model file or CPU extension), followed by ffmpeg and any of its related arguments.

In [0]:
python app.py -m “model.xml” | ffmpeg -framerate 24

* Set up Your Own Server on Linux

    https://opensource.com/article/19/1/basic-live-video-streaming-server

* Use Flask and Python

    https://www.pyimagesearch.com/2019/09/02/opencv-stream-video-to-web-browser-html-page/



#Handling Statistics and Images from a Node Server

Node.js is an open-source environment for servers, where Javascript can be run outside of a browser.

Node server can be used to handle the data coming in from the MQTT and FFmpeg servers, and then actually render that content for a web page user interface.



In [0]:
#import the MQTT Python library.
import paho.mqtt.client as mqtt


Need to import `socket` to connect to the MQTT server. Then, to get the IP address and set the port for communicating with the MQTT server

In [0]:
HOSTNAME = socket.gethostname()
IPADDRESS = socket.gethostbyname(HOSTNAME)
MQTT_HOST = IPADDRESS
MQTT_PORT = 3001
MQTT_KEEPALIVE_INTERVAL = 60

The real importance is here to make sure port number matches to what is set for the MQTT broker server to be listening on (Refer `config.js` within the MQTT server's files in the workspace).

In [0]:
#Connecting to the client can be accomplished with
client = mqtt.Client()
client.connect(MQTT_HOST, MQTT_PORT, MQTT_KEEPALIVE_INTERVAL)


In [0]:
#final piece for MQTT is to actually publish the statistics to the connected client

topic = "some_string"
client.publish(topic, json.dumps({"stat_name": statistic}))

The topic here should match to the relevant topic that is being subscribed to from the other end, while the JSON being published should include the relevant name of the statistic for the node server to parse (with the name like the key of a dictionary), with the statistic passed in with it (like the items of a dictionary).

In [0]:
client.publish("class", json.dumps({"class_names": class_names}))
client.publish("speedometer", json.dumps({"speed": speed}))

In [0]:
#at the end of processing the input stream, make sure to disconnect.
client.disconnect()

In [0]:
#FFmpeg does not actually have any real specific imports, although need the standard sys library
import sys

`ffserver` can be configured to read from `sys.stdout`. Once the output frame has been processed (drawing bounding boxes, semantic masks, etc.), you can write the frame to the `stdout` buffer and `flush` it.

In [0]:
sys.stdout.buffer.write(frame)  
sys.stdout.flush()

#Running the App

In [0]:
python app.py | ffmpeg -v warning -f rawvideo -pixel_format bgr24 -video_size 1280x720 -framerate 24 -i - http://0.0.0.0:3004/fac.ffm